# Backtesting with Fastquant

[Fastquant is a popular single trading pair backtesting library](https://github.com/enzoampil/fastquant).
In this example we will show how to use Fastquant to do a simple strategy backtest for a trading pair from Sushiswap.

We will

* Pick the SUSHI-USDT pair, just because we love the original food token

* Have $10,000 starting balance

* Trade with Simple Moving Average Crossover strategy

Internally Fastquant is based on [Backtrader](https://tradingstrategy.ai/docs/glossary.html#term-Backtrader).

## Creating trading universe

First let's initialise our dataset client.
This will automatically prompt you for a registration unless you already have an API key.

In [1]:
try:
    import tradingstrategy
except ImportError:
    %pip install trading-strategy
    import site
    site.main()

from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


Let's pick a single SUSHI-USDT pair for the strategy analysis.

In [2]:
from tradingstrategy.chain import ChainId
from tradingstrategy.pair import PandasPairUniverse

# Get the map of all supported decentralised exchanges
exchange_universe = client.fetch_exchange_universe()

# Fetch all trading pairs across all exchanges
columnar_pair_table = client.fetch_pair_universe()
pair_universe = PandasPairUniverse(columnar_pair_table.to_pandas())

# Pick SUSHI-USDT trading on SushiSwap
sushi_swap = exchange_universe.get_by_chain_and_slug(ChainId.ethereum, "sushiswap")
sushi_usdt = pair_universe.get_one_pair_from_pandas_universe(
    sushi_swap.exchange_id,
    "SUSHI",
    "USDT")

(Unknown total file size): 0it [00:00, ?it/s]

Fetch all daily candles and then filter

In [3]:
import datetime
import pandas as pd

from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.frameworks.backtrader import prepare_candles_for_backtrader

# Get daily candles as Pandas DataFrame
all_candles = client.fetch_all_candles(TimeBucket.d1).to_pandas()
sushi_usdt_candles: pd.DataFrame  = all_candles.loc[all_candles["pair_id"] == sushi_usdt.pair_id]

# Reformat data suitable for Backtrader based backtesting
sushi_usdt_candles = prepare_candles_for_backtrader(sushi_usdt_candles)

# To make this notebook deterministic, we pick date ranges for the test
start = datetime.datetime(2020, 9, 15)
end = datetime.datetime(2021, 7, 1)

sushi_usdt_candles = sushi_usdt_candles[
    (sushi_usdt_candles.index >= start) &
    (sushi_usdt_candles.index <= end)]

print(f"We have {len(sushi_usdt_candles)} daily candles for SUSHI-USDT")

(Unknown total file size): 0it [00:00, ?it/s]

We have 290 daily candles for SUSHI-USDT


## Creating and running Fastquant backtest

In this strategy, we will do a backtest for Simple Moving Average Crossover.
We use the following parameters:

* Slow window 10 days

* Fast window 20 days

See [documentation for SMAC](https://github.com/enzoampil/fastquant#simple-moving-average-crossover-15-day-ma-vs-40-day-ma).

In [4]:
from matplotlib.figure import Figure
from fastquant import backtest

result: pd.DataFrame
trade_graph: Figure

result, trade_graph = backtest(
    'smac',  # Simple Moving Average Crossover
    sushi_usdt_candles,
    init_cash=10_000, # Start with 10,000 USD
    fast_period=10,
    slow_period=20,
    return_plot=True,
    verbose=0)

<IPython.core.display.Javascript object>

## Analysing trades

Here are the key variables for the technical analysis of this strategy.

In [5]:
from tradingstrategy.frameworks.fastquant import to_human_readable_result
from IPython.display import display, HTML

# Get rid of extra columns, transpose the table
HTML(to_human_readable_result(result).to_html(header=False))

Sharpe ratio,2.904335
Maximum drawdown (%),72.15 %
Profit and loss (USD),"52,476.19"
Total returns (x),1.8322
Win rate (%),28.57 %
Won trades (n),2
Lost trades (n),5


## Trade graph

Below is the price and trade graph, showing when the trades happened.

In [6]:
from tradingstrategy.frameworks.matplotlib import render_figure_in_docs

render_figure_in_docs(trade_graph)

Onwards!